## Brief HowTo
- start the controller `ipcontroller --ip=<ip address>`
- copy the `ipcontroller-engine.json` to the correct folder on the engine machines (usually something like `/home/<user>/.ipython/profile_default/security/`)
- start the engines `ipcluster engines --n=<number of processes>`
- you can connect to the controller from a notebook on the same machine with `import ipyparallel as ipp; rc = ipp.Client()`
- executing `rc.ids` should show the ids of all connected engines

In [60]:
f = lambda x: x ** 4 - 3 * x

In [61]:
def integrate(f, a, b, n):
    dx = (b - a) / n
    s = 0.0
    x = a + 0.5 * dx
    for i in range(n):
        y = f(x)
        s += y
        x += dx
    return s * dx

In [134]:
ns = [1_000_000, 2_000_000, 4_000_000, 8_000_000, 12_000_000, 16_000_000, 20_000_000, 24_000_000]

In [135]:
import ipyparallel as ipp
rc = ipp.Client(profile="aspp")

In [144]:
rc.ids

[4, 5, 6, 7]

In [142]:
dview = rc[:]
dview.push({'f': f, 'integrate': integrate}).wait()

True

In [138]:
%%time
serial_result = list(map(lambda n: integrate(f, -2, 2, n), ns))
serial_result

CPU times: user 18 s, sys: 0 ns, total: 18 s
Wall time: 17.7 s


[12.799999999772574,
 12.799999999960521,
 12.799999999518676,
 12.800000000348296,
 12.800000002220681,
 12.800000002034913,
 12.799999996525015,
 12.800000003016375]

In [145]:
%%time
dview = rc[:]
parallel_result = dview.map_sync(lambda n: integrate(f, -2, 2, n), ns)
parallel_result

CPU times: user 98.2 ms, sys: 152 ms, total: 250 ms
Wall time: 9.85 s


[12.799999999772574,
 12.799999999960521,
 12.799999999518676,
 12.800000000348296,
 12.800000002220681,
 12.800000002034913,
 12.799999996525015,
 12.800000003016375]